In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [15]:
filename = '/content/drive/MyDrive/DSDojo/Colab Notebooks/Coding Dojo/Project 2/Project 2 cleaned final.csv'
df = pd.read_csv(filename, index_col='ID')
df.head()

,Still_Open,Art_Film_Music_1,Broadcasting_TV_2,Camps_Retrest_Confrc_3,Child_Adopt_Orphan_4,Chuch_Plant_5,Consult_Org&Tech_Support_6,Counceling_Rehab_7,Relig_Reform_Revival_8,Econom_Develop_9,Fellowship_Prof_Assoc_10,Health_Med_Serc_11,Housing_Cloth_Food_12,Interfaith_Dialog_13,Leader_Train_Edu_14,Legal_Political_Advocacy_15,Missions_16,Preach_Speak_Lect_17,Publishing_18,Other_19,Q3_ONE,Any_Category_Selec,Christian_NP,Jewish_NP,Muslim_NP,Hindu_NP,Buddhist_NP,IR_NP,Not_Reli_NP,Other_NP,Missing_Reli_Id_NP,Identifies_w_Denomination,Denom_Bible_Believ,Denom_Charismatic,Denom_Ecumenical,Denom_Evangelical,Denom_Fundamental,Denom_Mainline,Denom_Pentecostal,Denom_Theo_Conserv,...,Evangelical,Fundamentalist,Mainline,Pentecostal,Seeker,Theologically_Conservative,Theologically_Liberal,Other,Att_Relig_Serc,Wtinessed_Healing,Felt_Called,Had_Vision,Heard_Voice_God,Had_Dream,Had_Relig_Conv_Exper,Changed_Profoundly_Due_Relig_Exper,Region_1,Region_2,Region_3,Region_4,Region_5,Region_6,Region_7,Region_8,Region_9,Main_Category,Refused_Budget_Perc,Num_Budget_Spons,Perc_Finc_Indiv_Gifts_Don,Perc_Sales_Goods,Perc_Memberships,Perc_Spec_Fundraise,Perc_Grants_Don_Relig_Cong,Perc_Grants_Don_Denom_Off,Perc_Grants_Don_Relig_Found,Perc_Grant_Don_Sec_Found,Perc_Grant_Don_Bus_Corp,Perc_Grant_Gov,Perc_Other,Other.1
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11028,1.0,0,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,10.0,6,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,...,1,0,0,0,0,1,0,0,8,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,10,0,4,5.0,1.0,0,30,0,0,2,0,0,0.0,0,0
11051,1.0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,16.0,4,1,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,...,1,1,0,0,0,1,0,0,8,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,16,0,1,100.0,0.0,0,0,0,0,0,0,0,0.0,0,0
11055,1.0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,7.0,8,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,9,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,7,0,1,100.0,0.0,0,0,0,0,0,0,0,0.0,0,0
11056,1.0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,8.0,5,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8,0,1,100.0,0.0,0,0,0,0,0,0,0,0.0,0,0
11068,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,...,1,1,0,0,0,1,0,0,9,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,19,0,3,30.0,0.0,0,70,60,0,0,0,0,0.0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 919 entries, 11028 to 33188
Columns: 313 entries, Still_Open to Other.1
dtypes: float64(11), int64(300), object(2)
memory usage: 2.2+ MB


In [17]:
df.dtypes

Still_Open                  float64
Art_Film_Music_1              int64
Broadcasting_TV_2             int64
Camps_Retrest_Confrc_3        int64
Child_Adopt_Orphan_4          int64
                             ...   
Perc_Grant_Don_Sec_Found      int64
Perc_Grant_Don_Bus_Corp       int64
Perc_Grant_Gov              float64
Perc_Other                    int64
Other.1                       int64
Length: 313, dtype: object

We have some values in the target column that are blank. We are going to remove these and make them a 'fake' testing set. Then we will drop these from the original dataset

In [18]:
dfTestLater = df[df['Still_Open'].isnull()]


In [19]:
df = df[df['Still_Open'].notna()]


# Cleaning the Data

Check for duplicates

In [20]:
df.duplicated().value_counts()

False    901
dtype: int64

Drop Unnecessary columns:

In [21]:
dfNP =  df.drop(columns=['Q3_ONE','Any_Category_Selec','Q9_YES','Tot_Budget_Perc','Perc_Other_1','Other_1','Perc_Grants_Don_Relig_Cong1','Perc_Finc_Indiv_Gifts_Don1','Perc_Sales_Goods1','Perc_Memberships1','Perc_Spec_Fundraise1',
                         'Perc_Grants_Don_Denom_Off1','Perc_Grants_Don_Relig_Found1','Perc_Grant_Don_Sec_Found1','Perc_Grant_Don_Bus_Corp1','Perc_Grant_Gov1'])

In [22]:
dfNP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 901 entries, 11028 to 33188
Columns: 298 entries, Still_Open to Other.1
dtypes: float64(7), int64(289), object(2)
memory usage: 2.1+ MB


For Q24_1 (How many founders would you say this nonprofit had?) and Q24_2 (How many of these founders are still involved with this nonprofit?) we are going to replace the null values with the mode. This should not greatly effect the results as most NPs had between 1-3 founders so it makes sense to replace with the most common value instead of a 0 or an average.

In [23]:
dfNP['Num_Founders'].value_counts(dropna=False)

2.0      240
1.0      220
3.0      120
4.0       75
5.0       60
6.0       42
10.0      33
8.0       22
NaN       19
7.0       19
15.0      10
12.0       9
20.0       8
9.0        6
11.0       4
16.0       3
0.0        2
25.0       2
200.0      1
21.0       1
50.0       1
41.0       1
30.0       1
19.0       1
13.0       1
Name: Num_Founders, dtype: int64

In [24]:
numfound = dfNP['Num_Founders'].mode()[0]
numfoundinv = dfNP['Num_Founders_Still_Inv'].mode()[0]
dfNP['Num_Founders'] = dfNP['Num_Founders'].replace(np.nan,numfound)

In [25]:
dfNP['Num_Founders_Still_Inv'] = dfNP['Num_Founders_Still_Inv'].replace(np.nan,numfoundinv)

In [26]:
dfNP['Num_Founders_Still_Inv'].value_counts(dropna=False)

1.0      293
2.0      263
3.0      105
0.0       52
4.0       52
5.0       45
6.0       31
7.0       15
8.0       15
10.0      11
12.0       6
9.0        4
25.0       2
15.0       2
20.0       1
200.0      1
36.0       1
16.0       1
120.0      1
Name: Num_Founders_Still_Inv, dtype: int64

Check for inconistent datatypes

In [29]:
dfNP.select_dtypes(exclude='int')

,Still_Open,Q9_YES,Num_Founders,Num_Founders_Still_Inv,Yrs_Of_Training,Num_Hrs_PM_At_NP,Perc_Finc_Indiv_Gifts_Don,Perc_Sales_Goods,Perc_Grant_Gov
ID,,,,,,,,,
11028,1.0,NaN,1.0,1.0,12,200.0,5.0,1.0,0.0
11051,1.0,8 different countries,15.0,3.0,4,20.0,100.0,0.0,0.0
11055,1.0,Central America,1.0,1.0,2,30.0,100.0,0.0,0.0
11056,1.0,NaN,4.0,2.0,0,4.0,100.0,0.0,0.0
11068,1.0,Canada,4.0,0.0,0,100.0,30.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
33172,1.0,NaN,7.0,0.0,0,12.0,40.0,15.0,0.0
33176,1.0,NaN,3.0,1.0,0,50.0,75.0,10.0,0.0
33179,1.0,"""Canada, Australia, New Zealand""",3.0,1.0,0,160.0,50.0,10.0,0.0


Lets convert these to ints or floats where applicable


In [ ]:
dfNP['Still_Open']

# Code Sources:
[Fill NaN with Mode](https://stackoverflow.com/questions/40619445/how-to-replace-na-values-with-mode-of-a-dataframe-column-in-python/40621330)

[Select all columns of a certain datatype](https://cmdlinetips.com/2019/04/how-to-select-columns-by-data-type-in-pandas/)